# Allen Cahn equation

* Physical space 
\begin{align}
u_{t} = \epsilon u_{xx} + u - u^{3}
\end{align}
* Discretized with Chebyshev differentiation matrix (D)
\begin{align}
u_t = (\epsilon D^2 + I)u - u^{3}
\end{align}

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from rkstiff.solveras import SolverConfig
from rkstiff.etd import ETDConfig
from rkstiff.grids import construct_x_dx_cheb
from rkstiff.etd35 import ETD35
from rkstiff.etd34 import ETD34
from rkstiff.if34 import IF34
import time
import sys

# Linear operator, nonlinear function

In [ ]:
N = 20
epsilon = 0.01
x,D = construct_x_dx_cheb(N,-1,1)
D2 = D.dot(D)
L = epsilon*D2 + np.eye(*D2.shape)
L = L[1:-1,1:-1] # Interior points

def nl_func(u):
    return x[1:-1] - np.power(u+x[1:-1],3)

# Set initial field

In [ ]:
u0 = 0.53*x + 0.47*np.sin(-1.5*np.pi*x)
w0 = u0 - x

plt.plot(x,u0)
plt.xlabel('x')
plt.ylabel('$u_0$')
plt.title('Initial field')

# Apply nondiagonal and diagonalized solvers for comparison

In [ ]:
config = SolverConfig(epsilon=1e-4)
etd_config_nondiag = ETDConfig(contour_points=64, contour_radius=20)
solver34 = ETD34(lin_op=L, nl_func=nl_func, config=config, etd_config=etd_config_nondiag)
solver35 = ETD35(lin_op=L, nl_func=nl_func, config=config, etd_config=etd_config_nondiag)
solverIF = IF34(lin_op=L, nl_func=nl_func, config=config)

etd_config_diag = ETDConfig(contour_points=32, contour_radius=1)
solverDiag34 = ETD34(lin_op=L, nl_func=nl_func, config=config, etd_config=etd_config_diag, diagonalize=True)
solverDiag35 = ETD35(lin_op=L, nl_func=nl_func, config=config, etd_config=etd_config_diag, diagonalize=True)
solverDiagIF34 = IF34(lin_op=L, nl_func=nl_func, config=config, diagonalize=True)
solvers = [solver34, solver35, solverIF, solverDiag34, solverDiag35, solverDiagIF34]
titles = ['ETD34','ETD35','IF34','ETD34 Diagonalized','ETD35 Diagonalized','IF34 Diagonalized']

# Run simulations

In [ ]:
Xvec,Tvec,Uvec = [],[],[]
for solver in solvers:   
    _ = solver.evolve(w0[1:-1],t0=0,tf=100)
    U = []
    for wint in solver.u:
        w = np.r_[0,wint.real,0]
        u = w + x
        U.append(u)
    U = np.array(U)
    t = np.array(solver.t)
    T,X = np.meshgrid(t,x,indexing='ij')
    Xvec.append(X); Tvec.append(T); Uvec.append(U)

# Plot results

In [ ]:
fig = plt.figure(figsize=(16,12))
for i in range(6):
    ax = fig.add_subplot(2,3,i+1,projection='3d')
    ax.plot_wireframe(Xvec[i],Tvec[i],Uvec[i],color='black')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_zlabel('z')
    ax.set_facecolor('white')
    ax.grid(False)
    ax.set_title(titles[i])
    ax.view_init(elev=36,azim=-131)

# Time simulations

In [ ]:
start = time.time()
solver = ETD34(lin_op=L,nl_func=nl_func, config=config, etd_config=etd_config_nondiag)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)
end = time.time()
print(titles[0],'-> {:.2e}'.format(end-start))

start = time.time()
solver = ETD35(lin_op=L,nl_func=nl_func, config=config, etd_config=etd_config_nondiag)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)
end = time.time()
print(titles[1],'-> {:.2e}'.format(end-start))

start = time.time()
solver = IF34(lin_op=L,nl_func=nl_func, config=config)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)
end = time.time()
print(titles[2],'-> {:.2e}'.format(end-start))

start = time.time()
solverDiag = ETD34(lin_op=L,nl_func=nl_func, config=config, etd_config=etd_config_diag, diagonalize=True)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)
end = time.time()
print(titles[3],'-> {:.2e}'.format(end-start))

start = time.time()
solver = ETD35(lin_op=L,nl_func=nl_func, config=config, etd_config=etd_config_diag, diagonalize=True)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)
end = time.time()
print(titles[4],'-> {:.2e}'.format(end-start))

start = time.time()
solver = IF34(lin_op=L,nl_func=nl_func, config=config, diagonalize=True)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)
end = time.time()
print(titles[5],'-> {:.2e}'.format(end-start))

# Diagonalizing IF method has little impact on performance

In [ ]:
%%timeit
solver = IF34(lin_op=L, nl_func=nl_func, diagonalize=True)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)

In [ ]:
%%timeit
solver = IF34(lin_op=L,nl_func=nl_func, diagonalize=False)
_ = solver.evolve(w0[1:-1],t0=0,tf=100,store_data=False)